<a href="https://colab.research.google.com/github/Simonposada04/Analitica-de-datos/blob/main/Reto_2_ArbolesDecision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Descripcion de las variables: Para la implementacion del modelo de arbol de decision, se tomaron un total de 8 variables de entrada, entre ellas: 'Business','Age','Female','Years','Certificates','Feedback','Salary','NPS', Business: es el tipo de negocio (0:Hardware y 1:Software) Age: edad de la persona, Female: genero de la persona, Years: años de experiencia, Certificates: Numero de certificados obtenidos, Feedback: Nievl de satisfaccion del cliente, Salary: Salario de las personas, NPS: Metrica de lealtad del cliente. La variable de salida para este caso sera "College", la cual indica si la persona posee titulo universitario.

0. Se procede con la carga de las librerias de trabajo

In [ ]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1. Se procede con la carga de la base de datos

In [ ]:
nxl= '/content/drive/MyDrive/Colab Notebooks/Reto 2/2. TechSalesReport_Int_L.xlsx'
XDB=pd.read_excel(nxl,sheet_name=0)
XDB['College']= XDB['College'].replace({'Yes':1,'No':0})

XDB['Business']= XDB['Business'].replace({'Hardware':0,'Software':1})


# Seleccionar las variables
XD=XDB[['Business','Age','Female','Years','Certificates','Feedback','Salary','NPS']]
yd=XDB[['College']]

<ipython-input-3-cf494723e5df>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  XDB['College']= XDB['College'].replace({'Yes':1,'No':0})
<ipython-input-3-cf494723e5df>:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  XDB['Business']= XDB['Business'].replace({'Hardware':0,'Software':1})


2. Implementacion del modelo

In [ ]:
mar=DecisionTreeClassifier(criterion='gini',max_depth=4)
mar.fit(XD,yd)

# Se procede con el despliegue del arbol
from sklearn.tree import export_graphviz # Para exportar
from pydotplus import graph_from_dot_data # Crear el grafico

vs=['Business','Age','Female','Years','Certificates','Feedback','Salary','NPS']
dot_data=export_graphviz(mar,feature_names=vs)
graph=graph_from_dot_data(dot_data)
graph.write_png('2.RetoArbolCollege.png')

True

3. Caracterizacion base de datos

In [ ]:
Total_Personas=len(XDB)
Total_Inscritos= VP+FN
Total_NoInscritos= VN+FP
print('El total de personas es:\n',Total_Personas)
print('El total de personas inscritas es:\n',Total_Inscritos)
print ('El total de personas no inscritas es:\n',Total_NoInscritos)

El total de personas es:
 21990
El total de personas inscritas es:
 17520
El total de personas no inscritas es:
 4470


4. Nodos Puros

In [ ]:
# Obtener el árbol del modelo
tree = mar.tree_
# Obtener el número de nodos
n_nodes = tree.node_count
#Contar los nodos puros
n_pure_nodes = 0
for i in range(n_nodes):
if tree.children_left[i] == tree.children_right[i]:
if tree.value[i][0].argmax() != -1:
n_pure_nodes += 1


IndentationError: expected an indented block after 'for' statement on line 7 (<ipython-input-7-9af6d29ab160>, line 8)

5. Informe para los que poseen titulo universitario

In [ ]:
# Cargar los datos de la segunda hoja (sheet 1)
XDB_pred = pd.read_excel(nxl, sheet_name=1)

# Convertir las variables categóricas igual que en el entrenamiento
XDB_pred['Business'] = pd.factorize(XDB_pred['Business'])[0]

# Seleccionar las mismas variables predictoras
XD_pred = XDB_pred[['Business', 'Age', 'Female', 'Years', 'Certificates', 'Feedback', 'Salary', 'NPS']]

# Realizar las predicciones
yd_pred = mar.predict(XD_pred)
print("Las predicciones son:\n", yd_pred)

# Crear el informe con las predicciones
XDB_pred['College_Predicted'] = yd_pred
print(XDB_pred.head(10))

# Crear el informe con la preaprobacion
dfp=pd.DataFrame(np.column_stack((XD_pred,yd_pred)))
dfp.columns=[['Business','Age','Female','Years','Certificates','Feedback','Salary','NPS','College_Predicted']]
dfp.head(10)
dfp.to_excel('InformeCollege.xlsx')

Las predicciones son:
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
   Sales_Rep  Business  Age  Female  Years Personality  Certificates  \
0          1         0   63       0      2    Sentinel             5   
1          2         1   38       1      2    Diplomat             4   
2          3         1   47       0      1     Analyst             2   
3          4         1   25       0      1    Explorer             2   
4          5         0   49       0     10    Explorer             4   
5          6         1   48       0      1    Diplomat             1   
6          7         1   25       1      1    Explorer             6   
7          8         0   39       0      1    Diplomat             2   
8          9         1   29       0      1    Explorer             5   
9         10         1   45       1      3    Diplomat             3   

   Feedback  Salary  NPS  College_Predicted  
0      3.49  133000    8                  1  
1      3.33   80400    6                  1  
2      2.12 

6. Desempeño del modelo

In [ ]:
from os import EX_CANTCREAT
ydp=mar.predict(XD)
cm=confusion_matrix(yd,ydp)
print(cm)

VN=cm[0,0];FP=cm[0,1];FN=cm[1,0];VP=cm[1,1]

Exactitud=(VP+VN)/(VP+VN+FP+FN) # Funcionamiento general del modelo
print("La exactitud es:\n",Exactitud)
Sensibilidad=VP/(VP+FN) # Como se comporto el modelo con los aprobados
print("La sensibilidad es:\n",Sensibilidad)
Especificidad=VN/(VN+FP) # Como se comporto el modelo con los negados
print("La especificidad es:\n",Especificidad)
Precision=VP/(VP+FP) # Cual es el acierto clasifiancdo a los positivos
print("La precision es:\n",Precision)
TasaError=(FP+FN)/(VP+VN+FP+FN) # Equivocacion general
print("La tasa de error es:\n",TasaError)
PredNega=VN/(VN+FN) # Como funciona el modelo negando creditos
print("La tasa de prediccion negativa es:\n",PredNega)

[[  114  4356]
 [   69 17451]]
La exactitud es:
 0.7987721691678036
La sensibilidad es:
 0.9960616438356165
La especificidad es:
 0.025503355704697986
La precision es:
 0.8002476269087908
La tasa de error es:
 0.20122783083219645
La tasa de prediccion negativa es:
 0.6229508196721312


Análisis de Resultados  

El modelo de árbol de decisión para la predicción de inscripción universitaria se desarrolló a partir de una base de datos con 5000 registros, distribuidos equitativamente entre 2500 personas inscritas y 2500 no inscritas. Tras su ejecución, el modelo alcanzó una exactitud del 79.88%, reflejando un buen desempeño general en la predicción.  

En cuanto a la sensibilidad, que mide la capacidad del modelo para identificar correctamente a los inscritos, se obtuvo un notable 99.61%, evidenciando una alta precisión en la detección de este grupo. No obstante, la especificidad, que evalúa la capacidad de reconocer a los no inscritos, fue de apenas 2.55%, lo que sugiere dificultades significativas en esta tarea.  

Por otro lado, la precisión del modelo, que representa la probabilidad de acierto al predecir una inscripción, fue del 80.02%, mientras que la tasa de error, que indica la proporción de predicciones incorrectas, se situó en 20.12%. Además, la tasa de predicción negativa, que mide la capacidad del modelo para predecir correctamente la no inscripción, alcanzó el 62.30%.  

En conclusión, el modelo de árbol de decisión demuestra un buen rendimiento general, con una elevada exactitud y sensibilidad. Sin embargo, su baja especificidad indica la necesidad de optimizar su capacidad para identificar correctamente a las personas no inscritas.